In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data_path = "C:/Users/minkyu/Desktop/open/"
base_path = 'C:/Users/minkyu/Desktop/dacon accident prevention'

In [4]:
train = pd.read_csv(data_path+"train.csv" )
test = pd.read_csv(data_path+"test.csv" )
sample = pd.read_csv(data_path+"sample_submission.csv")

In [5]:
def preprocess(df):
    df.replace('-', np.nan, inplace=True)
    df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
    df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
    df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
    df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
    df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
    df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]
    df['사고인지 시간'] = df['사고인지 시간'].str.split('-').str[0].str.strip()

    return df


train = preprocess(train)
test = preprocess(test)


In [ ]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"'{row['공사종류(중분류)']}' 공사 중 "
            f"'{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [7]:
combined_training_data['question'][0]

"'건축물' 공사 중 '철근콘크리트공사' 작업에서 사고객체 '건설자재'(중분류: '철근')와 관련된 사고가 발생했습니다. 작업 프로세스는 '설치작업'이며, 사고 원인은 '고소작업 중 추락 위험이 있음에도 불구하고, 안전난간대, 안전고리 착용 등 안전장치가 미흡하였음.'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?"

In [8]:
# 훈련 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"'{row['공사종류(중분류)']}' 공사 중 "
            f"'{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [10]:
model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             quantization_config=bnb_config, 
                                             device_map="auto")

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.50s/it]


In [10]:
#gpu 강제 버전
model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0}  # GPU 0번에 모델 전체를 강제로 올림
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.54s/it]


In [11]:
# Train 데이터 준비
train_questions_prevention = combined_training_data['question'].tolist()
train_answers_prevention = combined_training_data['answer'].tolist()

train_documents = [
    f"Q: {q1}\nA: {a1}" 
    for q1, a1 in zip(train_questions_prevention, train_answers_prevention)
]

# 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-nli"  # 임베딩 모델 선택
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 벡터 스토어에 문서 추가
vector_store = FAISS.from_texts(train_documents, embedding)

# Retriever 정의
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

C:\Users\minkyu\AppData\Local\Temp\ipykernel_14104\1180258283.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)


In [12]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,  # sampling 활성화
    temperature=0.1,
    return_full_text=False,
    max_new_tokens=64,
)

prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 답변을 핵심 내용만 요약하여 간략하게 작성하세요.
- 서론, 배경 설명 또는 추가 설명을 절대 포함하지 마세요.
- 다음과 같은 조치를 취할 것을 제안합니다: 와 같은 내용을 포함하지 마세요.

{context}

### 질문:
{question}

[/INST]

"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  
    chain_type="stuff",  # 단순 컨텍스트 결합 방식 사용
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}  # 커스텀 프롬프트 적용
)

Device set to use cuda:0
C:\Users\minkyu\AppData\Local\Temp\ipykernel_14104\1051733876.py:26: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [14]:
# 테스트 실행 및 결과 저장
test_results = []

print("테스트 실행 시작... 총 테스트 샘플 수:", len(combined_test_data))

for idx, row in combined_test_data.iterrows():
    # 50개당 한 번 진행 상황 출력
    if (idx + 1) % 1 == 0 or idx == 0:
        print(f"\n[샘플 {idx + 1}/{len(combined_test_data)}] 진행 중...")

    # RAG 체인 호출 및 결과 생성
    prevention_result = qa_chain.invoke(row['question'])

    # 결과 저장
    result_text = prevention_result['result']
    test_results.append(result_text)

print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

테스트 실행 시작... 총 테스트 샘플 수: 964

[샘플 1/964] 진행 중...

[샘플 2/964] 진행 중...


KeyboardInterrupt: 

In [15]:
prevention_result

{'query': "'건축물' 공사 중 '철근콘크리트공사' 작업에서 사고객체 '건설기계'(중분류: '콘크리트펌프')와 관련된 사고가 발생했습니다. 작업 프로세스는 '타설작업'이며, 사고 원인은 '펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측 상부 1개소)가 발생하였고,  좌, 우측 아웃트리거의 펼친 길이가 상이하고 타설 위치가 건물 끝부분 모서리에 위치하여 붐대호스를 최대로 펼치다 보니 장비에 대한 무게중심이 한쪽으로 쏠려 일부 전도되는 사고가 발생된 것으로 판단됨'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?",
 'result': '### 답변:\n1. 펌프카 배치 시 지반 조사와 안정성 검토 강화.\n2. 아웃트리거 보강 구조물 설계 개선 및 현장 감독 강화.\n3. 펌프카 위치 선정 시 건물 구조와 주변 환경 고려',
 'source_documents': [Document(id='2381dc1b-4143-4670-b46c-80d14f970dfc', metadata={}, page_content="Q: '건축물' 공사 중 '철근콘크리트공사' 작업에서 사고객체 '시설물'(중분류: '건물')와 관련된 사고가 발생했습니다. 작업 프로세스는 '타설작업'이며, 사고 원인은 '1층 골조 타설을 위한 거푸집 지지용 동바리 하부(베이스)를 보강하고, 버팀대로 상호 연결하여, 동바리를 설치하였으나, 타설 전일 우천(1mm)으로 지표는 상태가 육안으로는 양호했으나, 타설 중 동바리가 일부 침하됨으로서, 사고부위 교실의 전체 동바리에 변형이 발생하여, 슬래브 침하 발생'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?\nA: 시공사 현장 정리 및 위험요소 점검 처리, 안전교육 철저, 공사장 부대 통로 연약지반 취환, 경상남도교육청 시설감리단 합동안전점검 및 공정회의 개최, 사고부위 재시공을 위한 구조기술사와 감독관 및 시공회사의 합동점검 실시."),
  Document(id='616e758e-c500-4428-8be

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

In [ ]:
submission = pd.read_csv(data_path+"sample_submission.csv", encoding = 'utf-8-sig')
# 최종 결과 저장
submission.iloc[:,1] = test_results
submission.iloc[:,2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('C:/Users/minkyu/Desktop/dacon accident prevention/baseline_submission.csv', index=False, encoding='utf-8-sig')